# Dynamic Wakes 1

## Wake propagation in heterogeneous backgrounds

In [ ]:
import foxes

In [ ]:
    mbook = foxes.models.ModelBook()
    mbook.wake_frames["dyn_wakes8"] = foxes.models.wake_frames.DynamicWakes(max_length_km=8)